In [1]:
import numpy as np
from modules import bayesianL
from modules import cubicharmonics
import matplotlib.pyplot as plt
import copy

In [2]:
#Carico i dati in data e li ordino in sdata e grid

data=np.transpose(np.loadtxt('../nlttk.out'))

sdata, grid = bayesianL.generatesorteddata(data, 500)

[1. 0. 0.] [0. 0. 1.] [0. 1. 0.]


In [3]:
#Medio i dati su tutti i punti equivalenti per simmetria cubica e ritorno la lista dei nuovi dati 'simmetrizzati'
#in gplot e sd
C=bayesianL.cubicarray(list(grid), True)
dic, dics=bayesianL.datadicG(data[1], data[2],cubicharmonics.Gvecgenerateall(500)[1:])
gplot, dataplot, datasigmaplot=bayesianL.datiplot(C, dic, dics)
sd=np.array([dataplot, datasigmaplot])

numero di elementi:  499
numero cicli teornicamente necessari 373501
numero cicli compiuti 381
numero di elementi non equivalenti per simmetria cubica:  118


In [10]:
#Calcolo il fit bayesiano con glot e sd come test
mN, SN, y_infer, sy_infer=\
bayesianL.bayesianpol(gplot,sd,12,50,0.001,gplot,ifprint=True, ifwarning=True)

parametri ottimali [  0.69119538 -20.51968815  48.29583745 134.06179027 -41.95409448
  83.59917249  -9.17087926  14.32510744  29.838686    -6.45114261
  21.26997882  17.22493013  10.41732134  -4.15744672 -25.86578643
  -3.3291239  -15.94041563 -11.62889009   2.80704879   0.61287846
  17.4492383  -12.47207582  10.97128482   6.98167629   6.67760203
  -3.19057852 -11.20310712   5.85864023  -0.87229789   3.46669865
  -3.98054304]
numero di armoniche cubiche 19
valore a kmin inferito 0.6019977645187617 bias 0.6911953843813565 dato a kmin 0.7147155658198961
determinante matrice delle armoniche cubiche ridotte: 5.774315552057058e+30


In [5]:
#Calcolo il best fit su gplot e sd con 30 dati
N=30
mN, SN, y_infer, sy_infer, spar, log_evidence_vP=\
bayesianL.bestfit(gplot, sd, N, gplot, ifprintbestfit=False,ifprintfinal= True)

parametri ottimali [  0.79678106 -16.78242705  28.84356    -14.46263304 -42.22790832
 -26.12954708 -14.49390261  -6.64882412 -31.38053117 -27.12617977
   5.68144126 -11.08960773   2.45805929  12.22583768   5.82442939
  13.43213524  -5.32105935  22.21029166  -3.86353939  -3.62151158
  -8.89353656  -5.20742178   5.48995209 -13.60055888   6.35635688
  -4.19053369   6.66282844  -4.07058737]
numero di armoniche cubiche 16
valore a kmin inferito 0.6558351602689977 bias 0.7967810601060614 dato a kmin 0.7147155658198961
determinante matrice delle armoniche cubiche ridotte: 2.079572965483931e+21
grado ottimale 22 grado massimo tentato 28


In [6]:
#Plotto la funzione di evidence che ho massimizzato per il best fit nella cella precedente
%matplotlib widget
plt.plot(log_evidence_vP, '.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
#Plotto il best fit e i dati (gplot e sd)
C=bayesianL.cubicarray(list(grid), True)
dic, dics=bayesianL.datadicG(data[1], data[2],cubicharmonics.Gvecgenerateall(500)[1:])
gplot, dataplot, datasigmaplot=bayesianL.datiplot(C, dic, dics)
%matplotlib widget

plt.fill_between(np.linalg.norm(gplot[:N], axis=1), y_infer+np.sqrt(sy_infer), y_infer-np.sqrt(sy_infer),  alpha=0.3, color='red')
plt.plot(np.linalg.norm(gplot[:N], axis=1), y_infer, '.-', color='red')
plt.errorbar(np.linalg.norm(gplot, axis=1), dataplot,datasigmaplot, fmt='.')

numero di elementi:  499
numero cicli teornicamente necessari 373501
numero cicli compiuti 381
numero di elementi non equivalenti per simmetria cubica:  118


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ErrorbarContainer object of 3 artists>

In [9]:
#Calcolo il best fit includendo sempre piu' punti 
predb=[]
spredb=[]
Npointsb=[]
for N in range(10, 114):
    
    mN, SN, y_infer, sy_infer, spar, log_evidence_vP=\
    bayesianL.bestfit(gplot, sd, N+1, gplot, ifprintbestfit=False, ifprintfinal=False)
    Npointsb.append(np.linalg.norm(gplot[N+1]))
    print(gplot[N+1], N+1)
    predb.append(mN[0])
    spredb.append(SN[0,0])

[2. 2. 2.] 11
[3. 2. 0.] 12
[1. 3. 2.] 13
[4. 0. 0.] 14
[0. 1. 4.] 15
[2. 3. 2.] 16
[0. 3. 3.] 17
[4. 1. 1.] 18
[3. 1. 3.] 19
[0. 2. 4.] 20
[2. 4. 1.] 21
[3. 2. 3.] 22
[2. 4. 2.] 23
[0. 5. 0.] 24
[0. 3. 4.] 25
[5. 1. 0.] 26
[1. 3. 4.] 27
[1. 5. 1.] 28
[3. 3. 3.] 29
[3. 2. 4.] 30
[0. 2. 5.] 31
[1. 2. 5.] 32
[4. 0. 4.] 33
[2. 5. 2.] 34


KeyboardInterrupt: 

In [13]:
#Calcolo il model averaging su gplot  e sd con 30 punti
N=20
mr, sr, logevidence=\
bayesianL.bayesianmodelprediction(gplot, sd, N, gplot, ifprintmodpred=False, ifprintfinal=True)

model average prediction 0.7556373989602236 0.012433362083034456


In [15]:
#Calcolo il model averaging includendo sempre piu' punti 
preda=[]
spreda=[]
Npointsa=[]
for N in range(10, 114):
    
    mr, sr, logevidence=\
    bayesianL.bayesianmodelprediction(gplot, sd, N+1, gplot, ifprintmodpred=False, ifprintfinal=True)
    Npointsa.append(np.linalg.norm(gplot[N+1]))
    print(gplot[N+1], N+1)
    preda.append(mr)
    spreda.append(sr)

model average prediction 0.5104902410789754 0.015593511811572373
[2. 2. 2.] 11
model average prediction 0.4715331150002475 0.015879093947785743
[3. 2. 0.] 12
model average prediction 0.4275853691170784 0.014843914862043277
[2. 1. 3.] 13
model average prediction 0.42645272717292276 0.015303970571119601
[0. 4. 0.] 14
model average prediction 0.38354397270055723 0.014677513134206852
[4. 1. 0.] 15
model average prediction 0.33278637249615284 0.012819575595280856
[2. 3. 2.] 16
model average prediction 0.29680243321973 0.014429935446985556
[1. 4. 1.] 17
model average prediction 0.2817951094989238 0.012683083840942357
[3. 3. 0.] 18


KeyboardInterrupt: 

In [20]:
%matplotlib widget
fig, ax=plt.subplots(3,constrained_layout=True, figsize=[9,9])


ax[0].set_title('Bayesian model averaging prediction at k=0 with data in range $[: k_{max}]$')

ax[0].errorbar(np.array(Npointsa), np.array(preda), np.sqrt(np.array(spreda)), fmt='.')
ax[0].plot(np.array(Npointsa), np.ones(len(Npointsa))*0.816, label='GK result')
    
ax[0].set_xlabel('$k_{max} (nm^{-1})$')
ax[0].set_ylabel('prediction at k=0')
ax[0].legend()
ax[1].set_title('Bayesian best model prediction at k=0 with data in range $[: k_{max}]$')

ax[1].errorbar(np.array(Npointsb), np.array(predb), np.sqrt(np.array(spredb)), fmt='.')
ax[1].plot(np.array(Npointsb), np.ones(len(Npointsb))*0.816, label='GK result')    
ax[1].set_xlabel('$k_{max} (nm^{-1})$')
ax[1].set_ylabel('prediction at k=0')
ax[1].legend()
ax[2].set_title('Data')

ax[2].errorbar(np.linalg.norm(gplot, axis=1), dataplot,datasigmaplot, fmt='.', label='3375 water molecules at 300k 30ns') 

ax[2].legend()

ax[2].set_xlabel(r'k ($nm^{-1}$)')
#plt.savefig('nlttcubic.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …